In [1]:
print("=" * 70)
print("BAGIAN 1: INSTALASI & KONFIGURASI")
print("=" * 70)

BAGIAN 1: INSTALASI & KONFIGURASI


In [2]:
print("\n📦 Instalasi Java, Kafka, dan Spark...")



📦 Instalasi Java, Kafka, dan Spark...


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/kafka/3.6.0/kafka_2.12-3.6.0.tgz
!tar -xzf kafka_2.12-3.6.0.tgz

In [5]:
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz


In [6]:
!pip install -q findspark kafka-python

print("✓ Instalasi selesai!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 kB 6.5 MB/s eta 0:00:00
✓ Instalasi selesai!


In [7]:
print("\n⚙️  Konfigurasi Environment Variables...")

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["KAFKA_HOME"] = "/content/kafka_2.12-3.6.0"

import findspark
findspark.init()

print("✓ Environment Variables siap!")


⚙️  Konfigurasi Environment Variables...
✓ Environment Variables siap!


In [9]:
print("\n🚀 Menjalankan Zookeeper dan Kafka Broker...")


🚀 Menjalankan Zookeeper dan Kafka Broker...


In [10]:
!nohup $KAFKA_HOME/bin/zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties > /dev/null 2>&1 &

import time
print("⏳ Menunggu Zookeeper start (10 detik)...")
time.sleep(10)

⏳ Menunggu Zookeeper start (10 detik)...


In [11]:
!nohup $KAFKA_HOME/bin/kafka-server-start.sh $KAFKA_HOME/config/server.properties > /dev/null 2>&1 &

print("⏳ Menunggu Kafka Broker start (10 detik)...")
time.sleep(10)

print("✓ Server Kafka siap digunakan!")


⏳ Menunggu Kafka Broker start (10 detik)...
✓ Server Kafka siap digunakan!


In [12]:
print("\n📋 Membuat Topic Kafka...")

!$KAFKA_HOME/bin/kafka-topics.sh --create --topic transaksi-toko --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

print("✓ Topik 'transaksi-toko' berhasil dibuat.")



📋 Membuat Topic Kafka...
Created topic transaksi-toko.
✓ Topik 'transaksi-toko' berhasil dibuat.


In [13]:
print("\n" + "=" * 70)
print("BAGIAN 2: KAFKA PRODUCER - DATA GENERATOR")
print("=" * 70)

from kafka import KafkaProducer
import json
import random
from datetime import datetime


BAGIAN 2: KAFKA PRODUCER - DATA GENERATOR


In [17]:
print("\n" + "=" * 70)
print("BAGIAN 2: KAFKA PRODUCER - DATA GENERATOR")
print("=" * 70)

from kafka import KafkaProducer
import json
import random
from datetime import datetime

# Fungsi untuk serialisasi data ke format JSON
def json_serializer(data):
    return json.dumps(data).encode("utf-8")

# Inisialisasi Producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=json_serializer
)

# Daftar produk dummy
products = ["Laptop", "Mouse", "Keyboard", "Monitor", "HDMI Cable"]

def send_stream_data(topic_name, num_messages=50):
    print(f"\n📤 Mulai mengirim {num_messages} data ke topik: {topic_name}...")
    for i in range(num_messages):
        # Membuat data transaksi palsu
        data = {
            "transaction_id": i,
            "product": random.choice(products),
            "price": random.randint(100000, 5000000),
            "quantity": random.randint(1, 5),
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        # Kirim ke Kafka
        producer.send(topic_name, data)

        # Simulasi jeda waktu antar transaksi (0.1 detik)
        time.sleep(0.1)

    print("✓ Selesai mengirim data.")


BAGIAN 2: KAFKA PRODUCER - DATA GENERATOR


In [18]:
print("\n" + "=" * 70)
print("BAGIAN 3: SPARK STRUCTURED STREAMING - CONSUMER")
print("=" * 70)

# Langkah 6: Inisialisasi Spark Session
print("\n⚡ Inisialisasi Spark Session...")

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, sum as _sum, count

spark = SparkSession.builder \
    .appName("KafkaSparkStreamingColab") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✓ Spark Session aktif.")



BAGIAN 3: SPARK STRUCTURED STREAMING - CONSUMER

⚡ Inisialisasi Spark Session...
✓ Spark Session aktif.


In [19]:
print("\n📖 Membaca Stream dari Kafka...")

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# 1. Definisi Schema
schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("product", StringType(), True),
    StructField("price", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("timestamp", StringType(), True)
])

# 2. Read Stream dari Kafka
df_raw = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "transaksi-toko") \
    .option("startingOffsets", "earliest") \
    .load()

# 3. Transformasi Data (Parsing JSON)
df_parsed = df_raw.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

print("✓ Schema data:")
df_parsed.printSchema()



📖 Membaca Stream dari Kafka...
✓ Schema data:
root
 |-- transaction_id: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- timestamp: string (nullable = true)



In [20]:
print("\n📊 Melakukan Agregasi Data...")

# Menambah kolom 'revenue' (price * quantity)
df_with_revenue = df_parsed.withColumn("revenue", col("price") * col("quantity"))

# Agregasi: Group by Product dan Sum Revenue
df_analysis = df_with_revenue.groupBy("product") \
    .agg(_sum("revenue").alias("total_sales")) \
    .orderBy("total_sales", ascending=False)



📊 Melakukan Agregasi Data...


In [21]:
print("🔄 Memulai Stream Processing...")

query = df_analysis.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("sales_table") \
    .start()

print("✓ Stream processing berjalan di background...")

🔄 Memulai Stream Processing...
✓ Stream processing berjalan di background...


In [22]:
print("\n" + "=" * 70)
print("BAGIAN 4: EKSEKUSI & MONITORING")
print("=" * 70)

# Langkah 10: Mengirim Data & Melihat Hasil
def run_analysis():
    # 1. Kirim 50 data baru ke Kafka
    send_stream_data("transaksi-toko", num_messages=50)

    print("\n" + "-" * 70)
    print("📈 HASIL ANALISIS DARI SPARK")
    print("-" * 70)

    # Tunggu sebentar agar Spark memproses data
    time.sleep(5)

    # 2. Query tabel hasil olahan Spark menggunakan SQL
    result = spark.sql("SELECT * FROM sales_table")
    result.show()

# Jalankan analisis pertama kali
run_analysis()

print("\n💡 Tip: Jalankan fungsi run_analysis() lagi untuk melihat data terus bertambah!")



BAGIAN 4: EKSEKUSI & MONITORING

📤 Mulai mengirim 50 data ke topik: transaksi-toko...
✓ Selesai mengirim data.

----------------------------------------------------------------------
📈 HASIL ANALISIS DARI SPARK
----------------------------------------------------------------------
+-------+-----------+
|product|total_sales|
+-------+-----------+
+-------+-----------+


💡 Tip: Jalankan fungsi run_analysis() lagi untuk melihat data terus bertambah!


In [23]:
print("\n" + "=" * 70)
print("TUGAS MANDIRI - MODIFIKASI ANALISIS")
print("=" * 70)

# TUGAS 1: Tambahkan Total Quantity per Produk
print("\n📝 TUGAS 1: Analisis dengan Total Sales + Total Quantity")
print("-" * 70)

# Stop query sebelumnya
query.stop()
time.sleep(2)

# Agregasi baru dengan Total Sales dan Total Quantity
df_analysis_extended = df_with_revenue.groupBy("product") \
    .agg(
        _sum("revenue").alias("total_sales"),
        _sum("quantity").alias("total_quantity")
    ) \
    .orderBy("total_sales", ascending=False)

# Start query baru
query_extended = df_analysis_extended.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("sales_extended_table") \
    .start()

print("✓ Query dengan Total Quantity dimulai...")

# Kirim data dan tampilkan hasil
send_stream_data("transaksi-toko", num_messages=50)
time.sleep(5)

print("\n📊 Hasil Analisis (Total Sales + Total Quantity):")
result_extended = spark.sql("SELECT * FROM sales_extended_table")
result_extended.show()



TUGAS MANDIRI - MODIFIKASI ANALISIS

📝 TUGAS 1: Analisis dengan Total Sales + Total Quantity
----------------------------------------------------------------------
✓ Query dengan Total Quantity dimulai...

📤 Mulai mengirim 50 data ke topik: transaksi-toko...
✓ Selesai mengirim data.

📊 Hasil Analisis (Total Sales + Total Quantity):
+-------+-----------+--------------+
|product|total_sales|total_quantity|
+-------+-----------+--------------+
+-------+-----------+--------------+



In [24]:
# TUGAS 2: Filter Transaksi dengan Price > 1.000.000
print("\n📝 TUGAS 2: Filter Transaksi Price > 1.000.000")
print("-" * 70)

# Stop query sebelumnya
query_extended.stop()
time.sleep(2)

# Tambahkan filter untuk price > 1000000
df_filtered = df_with_revenue.filter(col("price") > 1000000)

df_analysis_filtered = df_filtered.groupBy("product") \
    .agg(
        _sum("revenue").alias("total_sales"),
        _sum("quantity").alias("total_quantity"),
        count("*").alias("transaction_count")
    ) \
    .orderBy("total_sales", ascending=False)

# Start query dengan filter
query_filtered = df_analysis_filtered.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("sales_filtered_table") \
    .start()

print("✓ Query dengan filter Price > 1.000.000 dimulai...")

# Kirim data dan tampilkan hasil
send_stream_data("transaksi-toko", num_messages=50)
time.sleep(5)

print("\n📊 Hasil Analisis (Hanya Transaksi > 1 Juta):")
result_filtered = spark.sql("SELECT * FROM sales_filtered_table")
result_filtered.show()


📝 TUGAS 2: Filter Transaksi Price > 1.000.000
----------------------------------------------------------------------
✓ Query dengan filter Price > 1.000.000 dimulai...

📤 Mulai mengirim 50 data ke topik: transaksi-toko...
✓ Selesai mengirim data.

📊 Hasil Analisis (Hanya Transaksi > 1 Juta):
+-------+-----------+--------------+-----------------+
|product|total_sales|total_quantity|transaction_count|
+-------+-----------+--------------+-----------------+
+-------+-----------+--------------+-----------------+



In [25]:
# TUGAS 3 (OPSIONAL): Append Mode
print("\n📝 TUGAS 3 (OPSIONAL): Append Mode - Raw Transactions")
print("-" * 70)

# Stop query sebelumnya
query_filtered.stop()
time.sleep(2)

# Append mode: tampilkan raw transactions tanpa agregasi
query_append = df_with_revenue.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("transactions_raw") \
    .start()

print("✓ Query Append Mode dimulai...")

# Kirim beberapa data
send_stream_data("transaksi-toko", num_messages=20)
time.sleep(5)

print("\n📊 Raw Transactions (Append Mode):")
result_raw = spark.sql("SELECT transaction_id, product, price, quantity, revenue FROM transactions_raw LIMIT 20")
result_raw.show()



📝 TUGAS 3 (OPSIONAL): Append Mode - Raw Transactions
----------------------------------------------------------------------
✓ Query Append Mode dimulai...

📤 Mulai mengirim 20 data ke topik: transaksi-toko...
✓ Selesai mengirim data.

📊 Raw Transactions (Append Mode):
+--------------+----------+-------+--------+--------+
|transaction_id|   product|  price|quantity| revenue|
+--------------+----------+-------+--------+--------+
|             0|   Monitor|4678968|       3|14036904|
|             1|  Keyboard|1573421|       1| 1573421|
|             2|HDMI Cable|2649274|       5|13246370|
|             3|  Keyboard|2881302|       4|11525208|
|             4|   Monitor|2973933|       4|11895732|
|             5|     Mouse| 511946|       5| 2559730|
|             6|     Mouse|2710600|       2| 5421200|
|             7|HDMI Cable|1016730|       5| 5083650|
|             8|    Laptop| 471640|       1|  471640|
|             9|    Laptop|4173357|       4|16693428|
|            10|   Monitor|4

In [26]:
print("\n" + "=" * 70)
print("MEMBERSIHKAN RESOURCES")
print("=" * 70)



MEMBERSIHKAN RESOURCES


In [27]:
# Langkah 11: Menghentikan Program
query_append.stop()
print("✓ Streaming dihentikan.")

print("\n" + "=" * 70)
print("✅ PRAKTIKUM SELESAI!")
print("=" * 70)

print("\n💡 Ringkasan yang telah dikerjakan:")
print("   ✓ Setup Kafka & Spark di Google Colab")
print("   ✓ Membuat Producer untuk mengirim data dummy")
print("   ✓ Membuat Consumer dengan Spark Structured Streaming")
print("   ✓ TUGAS 1: Analisis Total Sales + Total Quantity")
print("   ✓ TUGAS 2: Filter transaksi Price > 1.000.000")
print("   ✓ TUGAS 3: Implementasi Append Mode")

# Uncomment jika ingin menghentikan Spark Session
# spark.stop()

✓ Streaming dihentikan.

✅ PRAKTIKUM SELESAI!

💡 Ringkasan yang telah dikerjakan:
   ✓ Setup Kafka & Spark di Google Colab
   ✓ Membuat Producer untuk mengirim data dummy
   ✓ Membuat Consumer dengan Spark Structured Streaming
   ✓ TUGAS 1: Analisis Total Sales + Total Quantity
   ✓ TUGAS 2: Filter transaksi Price > 1.000.000
   ✓ TUGAS 3: Implementasi Append Mode
